In [ ]:
import os
os.chdir('/home/fsboedij/Data/MouseAtlas')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
import seaborn as sns
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(5, 5))
torch.set_printoptions(precision=10, sci_mode=False, edgeitems=12)
pd.options.mode.chained_assignment = None
import matplotlib.colors as mcolors
import utils
from sklearn.linear_model import LinearRegression
import matplotlib.cm as cm


In [ ]:
adata=sc.read('Data/Integration/Step10_fulladata_final_0411.h5ad')
print(adata)

In [ ]:
#Celltype composition
import matplotlib.pyplot as plt
adata.obs['age_group']=adata.obs.age.copy()
adata.obs['age_group']=adata.obs['age_group'].astype(object)
adata.obs['age_group'].loc[adata.obs['age_group'].isin(['1','5-6','6-8','8','10-12','12'])] = 'Young'
adata.obs['age_group'].loc[adata.obs['age_group'].isin(['12-13','6-20','13','14','16-20'])] = 'Adult'
adata.obs['age_group'].loc[adata.obs['age_group'].isin(['18-20','78','104'])] = 'Old'

adata.obs['Level_3_5']=adata.obs['Level_3'].copy()
adata.obs['Level_3_5']=adata.obs['Level_3_5'].astype(object)
adata.obs['Level_3_5'].loc[(adata.obs['Level_3'].isin([ 'T cell lineage']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Mast cells']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Club_Goblet']))] = adata.obs.Level_4
adata.obs['Level_3_5'].loc[(adata.obs['Level_4'].isin([ 'Smoke induced club_goblet']))] = 'Club_Goblet'
adata.obs['Level_3_5'].loc[(adata.obs['Level_3'].isin([ 'Capillary endothelial','Venous endothelial']))] = 'Endothelial cells'



healthy=adata[adata.obs.healthy_or_disease=='Healthy'].copy()
healthy=healthy[healthy.obs.study!='Mouse_Post_Sendai'].copy()

desired_order = ['Young', 'Adult', 'Old']

# Ensure 'age_group' is a categorical variable with the specified order
healthy.obs['age_group'] = pd.Categorical(
    healthy.obs['age_group'],
    categories=desired_order,
    ordered=True
)

# Copying adata.obs to a DataFrame
df = healthy.obs.copy()

# Define the groups to iterate over
groups = ['sex', 'age_group', 'strain', 'platform']

for g in groups:
    print(g)
    # Group by the current group and 'Level_3', then count occurrences
    grouped_data = df.groupby([g, 'Level_3_5']).size().unstack(fill_value=0)

    # Convert counts to percentages
    percentage_data = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100
    percentage_data.to_csv(f'Data/others/celltypecomposition/celltypes_{g}.csv')


    # Plot a stacked bar graph with percentages
    ax = percentage_data.plot(
        kind='bar', 
        stacked=True, 
        figsize=(12, 8),  # Adjusted width for a wider figure
        colormap='tab20'
    )

    # Remove the grid
    ax.grid(False)

    # Customize the plot
    plt.title(f'Percentage Distribution of Level_3_5 by {g}', fontsize=14)
    ax.set_xlabel(f'{g}', fontsize=12)
    ax.set_ylabel('Percentage (%)', fontsize=16)
    ax.tick_params(axis='x', labelsize=24)

    # Rotate and align x-tick labels
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.legend(title='Level_3', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    plt.subplots_adjust(left=0.1,right=0.9,top=0.9,bottom=0.2)
    

    # Save the plot with increased width
    output_path = f"figures/celltypecomposition_{g}_level3_5.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')

    # Show the plot
    plt.show()

    #Table of each values per sample for stats
    # Group by 'sex', 'sample', and 'Level_3_5' and count occurrences
    grouped_data = df.groupby([g, 'sample', 'Level_3_5']).size().unstack(fill_value=0)
    percentage_data = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100
    percentage_data=percentage_data.fillna(0)
    percentage_data=percentage_data.transpose()
    percentage_data=percentage_data.loc[:,percentage_data.sum()!=0]
    # Display the table
    print(percentage_data)

    # Optionally, save the table to a CSV file
    percentage_data.to_csv(f'Data/others/celltypecomposition/Level_3_5_counts_by_{g}_and_sample.csv')
